# 필요한 라이브러리 로드

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import pandas as pd
from time import sleep
from tqdm import tqdm

# 초기 화면 로드

In [2]:
keyword = '프로미스나인+fromis_9'
url = 'https://www.youtube.com/results?search_query={}'.format(keyword)
s = Service('./chromedriver.exe')
driver = webdriver.Chrome(service=s)
driver.get(url)
action = ActionChains(driver)

# 동영상만 보기 설정

In [3]:
filter_element = WebDriverWait(driver, 10).until(
 EC.presence_of_all_elements_located((By.XPATH, "//*[@id='text']"))
)

filter_element[2].click()

In [4]:
filter_sub_element = WebDriverWait(driver, 10).until(
 EC.presence_of_all_elements_located((By.XPATH, "//*[contains(text(),'동영상')]"))
)

filter_sub_element[1].click()

# 스크롤 100회 내리기

In [5]:
for i in tqdm(range(100)):
    start = i * 10000
    end = (i+1) * 10000
    driver.execute_script(f"window.scrollTo({start},{end})")
    sleep(2)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [03:23<00:00,  2.03s/it]


# 제목, 업로더 크롤링

## 제목, 업로더 태그 가져오기

In [6]:
fromis_9_titles = driver.find_elements(By.ID, "video-title")
fromis_9_uploader= driver.find_elements(By.XPATH, "//a[@class='yt-simple-endpoint style-scope yt-formatted-string']")

## 제목, 업로더 텍스트 저장

In [7]:
titles = [x.text for x in fromis_9_titles]
uploaders = [fromis_9_uploader[i].text for i in fromis_9_uploader if i != '']

# 조회수, 날짜 크롤링

In [8]:
error_row = []

In [20]:
fromis_9_list = []
views = []
update_dates = []
    
for i,title in tqdm(enumerate(fromis_9_titles)):
    try:
        
        title_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable(title)
        )
        
        title_element.send_keys(Keys.CONTROL +"\n")
        driver.switch_to.window(driver.window_handles[1])
        
        view_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//span[@class='view-count style-scope ytd-video-view-count-renderer']"))
        )
        
        update_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//div[@id='info-strings']"))
        )
        
        views.append(view_element.text)
        update_dates.append(update_element.text)

        driver.close()

        # 메인 탭으로 이동
        driver.switch_to.window(driver.window_handles[0])
        
    except:
        error_row.append(i)
        print('오류')
        views.append('오류')
        update_dates.append('오류')
        driver.close()

        # 메인 탭으로 이동
        driver.switch_to.window(driver.window_handles[0])

529it [14:56,  1.69s/it]


In [21]:
error_row

[]

In [29]:
for title,uploader,view,update_date in zip(titles,uploaders,views,update_dates):
    fromis_9_list.append([title,uploader,view,update_date])

In [30]:
fromis_9_df = pd.DataFrame(data=fromis_9_list,columns=['제목','업로더','조회수','게시일'])

In [31]:
fromis_9_df = fromis_9_df.drop_duplicates(keep='first')

In [32]:
fromis_9_df=  fromis_9_df.reset_index(drop=True)

In [33]:
fromis_9_df

,제목,업로더,조회수,게시일
0,[CHANNEL_9] fromis_9 (프로미스나인) '채널나인' EP.20 RUN...,Official fromis_9,"조회수 62,823회",2021. 12. 30.
1,fromis_9 (프로미스나인) 'Talk & Talk' Official MV,HYBE LABELS,"조회수 21,665,354회",2021. 9. 1.
2,프로미스나인(Fromis_9) - Talk & Talk 교차편집(stage mix),강민혁 X 차누,"조회수 544,393회",2021. 9. 30.
3,[BE ORIGINAL] fromis_9 (프로미스나인) 'Talk & Talk' ...,STUDIO CHOOM [스튜디오 춤],"조회수 1,836,439회",2021. 9. 4.
4,[릴레이댄스] 프로미스나인 (fromis_9) - Talk & Talk (4K),M2,"조회수 605,471회",2021. 9. 5.
...,...,...,...,...
497,메리 규리스마스! ⛄ [프로미스나인],먐먀미스나인,조회수 468회,2021. 12. 27.
498,fromis_9 (프로미스나인) | Midnight Guest - Concept T...,GermanSone,조회수 269회,2021. 12. 30.
499,[MPD직캠] 프로미스나인 이채영 직캠 'Talk & Talk' (fromis_9 ...,Moniklodstudio MKS,조회수 42회,2021. 12. 27.
500,211224 네이버 NOW. (studio문나잇) 사녹 #프로미스나인 #fromis...,전람회 [ appleboy tv ],조회수 110회,2021. 12. 25.


In [34]:
fromis_9_df.to_csv('./data/fromis_9_youtube.csv',sep=',',na_rep='NaN',encoding='utf-8-sig')